In [1]:
import mne
import matplotlib.pyplot as plt
import PyQt5
import seaborn as sns
import numpy as np
import pandas as pd
import glob
from scipy import io, stats

In [2]:
plt.switch_backend('QtAgg')

In [3]:
def load_eeg(cnt_data, mrk_data, session = 1):
    return { 
        "clab": [_[0] for _ in cnt_data["cnt"][0,session-1]["clab"][0,0][0]],
        "fs": cnt_data["cnt"][0,session-1]["fs"][0,0][0,0],
        "x": pd.DataFrame(cnt_data["cnt"][0,session-1]["x"][0,0], columns=[_[0] for _ in cnt_data["cnt"][0,session-1]["clab"][0,0][0]]) * 1e-6, # conversion from 'uV' to 'V'
        "time": mrk_data["mrk"][0,session-1]["time"][0,0][0] * 1e-3,    # conversion from 'ms' to 's'
        "y": mrk_data["mrk"][0,session-1]["y"][0,0],
        "event": [_[0] for _ in mrk_data["mrk"][0,session-1]["event"][0,0][0,0][0]],
        "className": [_[0] for _ in mrk_data["mrk"][0,session-1]["className"][0,0][0]]
    }

In [4]:
subject = '04'              # subject id to determine file path

tmin, tmax = -5.0, 20.0     # epoch start/end relative to event marker (seconds)
baseline = (None, -2.0)     # baseline correction  
sfreq = 200                 # sampling frequency of dataset

montage = mne.channels.make_standard_montage("standard_1005")   # international 10-5 system

ch_names = ['F7', 'AFF5h', 'F3', 'AFp1', 'AFp2', 'AFF6h', 'F4', 'F8', 'AFF1h', 'AFF2h', 'Cz', 'Pz', 'FCC5h', 'FCC3h', 'CCP5h', 'CCP3h', 'T7', 'P7', 'P3', 'PPO1h', 'POO1', 'POO2', 'PPO2h', 'P4', 'FCC4h', 'FCC6h', 'CCP4h', 'CCP6h', 'P8', 'T8', 'VEOG', 'HEOG']   # all data channels

ch_types = ['eeg']*30 + ['eog']*2   # first 30 are EEG channels and other 2 are EOG channels

In [5]:
path = glob.glob(f'../dataset\\EEG_[0-2][0-9]-[0-2][0-9]\\subject {subject}\\with occular artifact\\')[0]
path

'../dataset\\EEG_01-05\\subject 04\\with occular artifact\\'

In [6]:
%%capture

raws = []

cnt = io.loadmat(path + 'cnt.mat')
mrk = io.loadmat(path + 'mrk.mat')

# session 1,3,5 for Dataset A (lhmi/rhmi)
# session 2,4,6 for Dateset B (ma/baseline)

for session in [1,3,5]:
    task = load_eeg(cnt, mrk, session)

    sfreq = task["fs"]  # 200Hz
    data = task["x"].loc[:, ch_names].values.T # ~ 120,000 samples and ~ 600 seconds
    
    onset = task["time"]
    duration = [tmax - tmin] * len(task["time"])
    description = list(map(lambda y: "cond1" if y else "cond2", task["y"][0]))

    # For Dataset A => cond1 = lhmi and cond2 = rhmi
    # For Dataset B => cond1 = ma and cond2 = baseline

    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    annotations = mne.Annotations(onset=onset, duration=duration, description=description)

    raw = mne.io.RawArray(data=data, info=info)
    raw.set_montage(montage)

    raw.set_annotations(annotations)

    raws.append(raw)

raw = mne.concatenate_raws(raws)

In [7]:
raw.plot(block=True)

Using matplotlib as 2D backend.
Channels marked as bad:
none


<MNEBrowseFigure size 800x800 with 4 Axes>

In [8]:
# raw.plot_sensors(kind='3d', ch_type='eeg', block=True)

In [9]:
%%capture

mne.set_eeg_reference(raw, ref_channels="average", ch_type='eeg', projection=False)    # common average reference

In [10]:
%%capture

raw_unfiltered = raw.copy()

raw.filter(picks="eeg", l_freq=1, h_freq=40)     # band pass filters

fig, ax = plt.subplots(2)

raw_unfiltered.plot_psd(ax=ax[0], show=False)
raw.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')

fig.set_tight_layout(True)

plt.show()  # block execution and analyze plots

In [11]:
%%capture

ica = mne.preprocessing.ICA(
    n_components=0.999,
    max_iter=500,
    method='picard',
    fit_params=dict(ortho=True, extended=True),
    random_state=42)

ica.fit(raw)

In [12]:
ica.get_explained_variance_ratio(raw)

{'eeg': 0.9979896426868188}

In [13]:
%%capture

eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None, baseline=(None, 0), tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(eog_epochs)

ica.exclude = eog_inds

In [14]:
ica.exclude

[0, 6]

In [15]:
%%capture

ica.apply(raw)  # Apply ICA

In [16]:
raw.plot(block=True)

Channels marked as bad:
none


<MNEBrowseFigure size 800x800 with 4 Axes>

In [17]:
%%capture

raw_unfiltered = raw.copy()

raw.filter(picks="eeg", l_freq=8, h_freq=30)     # band pass filters

fig, ax = plt.subplots(2)

raw_unfiltered.plot_psd(ax=ax[0], show=False)
raw.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')

fig.set_tight_layout(True)

plt.show()  # block execution and analyze plots

In [18]:
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: ['cond1', 'cond2']


In [19]:
epochs = mne.Epochs(
    raw, 
    events, 
    event_id=event_id, 
    tmin=tmin, 
    tmax=tmax, 
    baseline=baseline,
    reject_by_annotation=True)

Not setting metadata
60 matching events found
Setting baseline interval to [-5.0, -2.0] s
Applying baseline correction (mode: mean)
0 projection items activated


In [20]:
epochs.save(f"epochs/subject_{subject}_epo.fif", overwrite=True)

Using data from preloaded Raw for 60 events and 5001 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 5001 original time points ...
Using data from preloaded Raw for 60 events and 5001 original time points ...
